# Boundary Element Method

In [1]:
import numpy as np
import sympy as sp
import math as mt
import scipy.integrate as integrate
from tabulate import tabulate

<b>Boundaries</b>

In [38]:
def x11(t: float):
    return sp.cos(t)

def x12(t: float):
    return sp.sin(t)

def x21(t: float):
    return sp.cos(t) / 2

def x22(t: float):
    return sp.sin(t) / 2

<b>Derivatives</b>

In [3]:
t = sp.Symbol('t')
j = sp.Symbol('j')

In [4]:
def evalf_d(expr, tau: float, j: int):
    xj = expr.evalf(subs = {t: j * h})
    return xj + tau * (expr.evalf(subs = {t: (j + 1) * h}) - xj) 

In [5]:
def d_xj(x, tau: float, j: int):
    expr = sp.diff(x(t))
    return evalf_d(expr, tau, j)

In [6]:
def d2_xj(x, tau: float, j: int):
    expr = sp.diff(sp.diff(x(t)))
    return evalf_d(expr, tau, j)

In [7]:
def d_x1(t: float, j: int):
    return [d_xj(x11, t, j), d_xj(x12, t, j)]

def d_x2(t: float, j: int):
    return [d_xj(x21, t, j), d_xj(x22, t, j)]

<b>Integrals</b>

In [19]:
def quad(K):
    return integrate.quad(K, 0, 1)[0] / (2 * np.pi)

<b>Normals</b>

In [9]:
def v1(t: float, j: int):
    z = sp.sqrt(np.dot(x1(t, j), x1(t, j)))
    return [xj(x12, t, j) / z, - xj(x11, t, j) / z]

def v2(t: float, j: int):
    z = sp.sqrt(np.dot(x2(t, j), x2(t, j)))
    return [xj(x22, t, j) / z, - xj(x21, t, j) / z]

<b>Elements Count</b>

In [86]:
N = 8 # count
h = 1 / N # step

<b>Aproximation with Linear Function</b>

In [11]:
def xj(x, t: float, j: int):
    xj = x(j * h) + t * (x((j + 1) * h) - x(j * h))
    return xj

def x1(t: float, j: int):
    return [xj(x11, t, j), xj(x12, t, j)]

def x2(t: float, j: int):
    return [xj(x21, t, j), xj(x22, t, j)]

<b>Kernels</b>

In [111]:
def K11(t: float, tau: float, j: int, k: int):
    if t != tau:
        sub = np.subtract(x1(t, k), x1(tau, j))
        return ((xj(x11, t, k) - xj(x11, tau, j)) * d_xj(x12, tau, j) - (xj(x12, t, k) - xj(x12, tau, j)) * d_xj(x11, tau, j)) / np.dot(sub, sub)
    else:
        return (d_xj(x11, t, k) * d2_xj(x12, t, k) - d_xj(x12, t, k) * d2_xj(x11, t, k)) / (2 * np.dot(d_x1(t, k), d_x1(t, k)))
    
def K12(t: float, tau: float, j: int, k: int):
    sub = np.subtract(x1(t, k), x2(tau, j))
    return mt.log(1 / mt.sqrt(np.dot(sub, sub))) * mt.sqrt(np.dot(d_x1(tau, j), d_x1(tau, j)))

def K21(t: float, tau: float, j: int, k: int):
    sub = np.subtract(x2(t, k), x1(tau, j))
    return (np.dot(v2(t, k), v1(tau, j)) / np.dot(sub, sub) - 2 * np.dot(sub, v2(t, k)) * np.dot(sub, v1(tau, j)) / np.dot(sub,sub) ** 2) * mt.sqrt(np.dot(d_x2(tau, j), d_x2(tau, j)))

def K22(t: float, tau: float, j: int, k: int):
    if t != tau:
        sub = np.subtract(x2(t, k), x2(tau, j))
        return ((xj(x21, t, k) - xj(x21, tau, j)) * d_xj(x22, t, k) - (xj(x22, t, k) - xj(x22, tau, j)) * d_xj(x21, t, k)) / np.dot(sub, sub)
    else:
        return - (d_xj(x21, t, k) * d2_xj(x22, t, k) - d_xj(x22, t, k) * d2_xj(x21, t, k)) / (2 * np.dot(d_x2(t, k), d_x2(t, k)))

In [114]:
def matrix_A():
    matrix = np.zeros((2 * N, 2 * N))
    
    for k in range(0, N):
        
        print("Перший квадрат")
        for j in range(0, N):
            matrix[k][j] = quad(lambda tau: K11(1 / 2, tau, j, k))
            if k == j:
                matrix[k][j] -= 1 / 2
        
        j = 0
        print("Другий  квадрат")
        for i in range(N, 2 * N):
            matrix[k][i] = quad(lambda tau: K12(1 / 2, tau, j, k))
            j += 1

    k = 0
    for q in range(N, 2 * N):
        
        print("Третій квадрат")
        for j in range(0, N):
            matrix[q][j] = quad(lambda tau: K21(1 / 2, tau, j, k))
            
        print("Четвертий квадрат")
        j = 0
        for i in range(N, 2 * N):
            matrix[q][i] = quad(lambda tau: K22(1 / 2, tau, j, k))
            if q == i:
                matrix[q][i] -= 1 / 2 
            j += 1
        k += 1
            
    print(matrix)
    return matrix

In [85]:
def vector_b():
    vec_b = np.zeros((2 * N, 1))
    
    for i in range(0, N):
        vec_b[i][0] = 1
        
    for i in range(N, 2 * N):
        vec_b[i][0] = 0
        
    print(vec_b)
    return vec_b

In [109]:
def solve_system(A, b):
    m = np.linalg.solve(A, b)
    print(m)
    
    u = 0

    for i in range(0, len(m)):
        u += m[i][0]

    return u

In [115]:
A = matrix_A()

Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
[[-0.65915494 -0.09043163 -0.08144812 -0.08036223 -0.08001319 -0.07985677
  -0.07977317 -0.07972324  0.1103812   0.10795401  0.10111796  0.09097903
   0.07882648  0.06576439  0.05257898  0.03976414]
 [-0.09043163 -0.65915494 -0.09043163 -0.08144812 -0.08036223 -0.08001319
  -0.07985677 -0.07977317  0.10795401  0.1103812   0.10795401  0.10111796
   0.09097903  0.07882648  0.06576439  0.05257898]
 [-0.08144812 -0.09043163 -0.65915494 -0.09043163 -0.08144812 -0.08036223
  -0.08001319 -0

In [88]:
print(tabulate(A))
print("det = " + str(np.linalg.det(A)))

-----------  -----------  -----------  ----------  ----------  -----------  -----------  -----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------
-0.659155    -0.0904316   -0.0814481   -0.0803622  -0.0800132  -0.0798568   -0.0797732   -0.0797232    0.110381    0.107954    0.101118    0.090979    0.0788265   0.0657644   0.052579    0.0397641
-0.0904316   -0.659155    -0.0904316   -0.0814481  -0.0803622  -0.0800132   -0.0798568   -0.0797732    0.107954    0.110381    0.107954    0.101118    0.090979    0.0788265   0.0657644   0.052579
-0.0814481   -0.0904316   -0.659155    -0.0904316  -0.0814481  -0.0803622   -0.0800132   -0.0798568    0.101118    0.107954    0.110381    0.107954    0.101118    0.090979    0.0788265   0.0657644
-0.0803622   -0.0814481   -0.0904316   -0.659155   -0.0904316  -0.0814481   -0.0803622   -0.0800132    0.090979    0.101118    0.107954    0.110381    0.107954    0.101118    0.090979    0.0788265
-0.0800132   -0.

In [116]:
b = vector_b()
print(solve_system(A, b))

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[-0.23613935]
 [-0.08285729]
 [ 0.02172773]
 [ 0.07584152]
 [ 0.07584152]
 [ 0.02172773]
 [-0.08285729]
 [-0.23613935]
 [ 1.25420661]
 [ 1.25157782]
 [ 1.28624749]
 [ 1.3144578 ]
 [ 1.3144578 ]
 [ 1.28624749]
 [ 1.25157782]
 [ 1.25420661]]
9.770124689862033
